## Modelo 3: 
### Otro approach distinto.

Con la idea en mente de entrenar de cero la red, se realizará la siguiente arquitectura de CNN para clasificar la basura.

Previamente se habia utilizado un modelo preentrenado, sobre el cual se sobreentreno el dataset actual, y se cometió el error de utilizar como parte del dataset de validación, elementos del dataset de entrenamiento. Por lo que no se podian observar cuales eran los errores asociados en la etapa de validación. 

Para generar el modelo utilizaremos PyTorch y su libreria de redes neuronales, y nos enfocaremos en crear un clasificador que tiene como input imagenes de 160x120x3 pixeles, y a través de filtros convolucionales y un clasificador lineal con una softmax en la salida, generar una districuición probabilidstica que determine cual de las 3 clases de salida, Botella, Lata, Tetrapak, es la más probable dada la anterior.

### Importamos las librerias correspondientes:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

### Importaremos las imagenes a clasificar.

Por lo anterior, se utilizará un approach más común el cual es dividir el dataset de entrenamiento en 3 dataset, randomizado por una semilla.
El primero, es el nuevo dataset de entrenamiento, consistirá de alrededor de un 60% de las imagenes originales, luego un dataset de validacion, con 20% del dataset original, y el 20% restante es para probar finalmente el rendimiento del clasificador. 

Partamos importando las imagenes a clasificar, del dataset reducido, con resolución de 160x120px, con 3 canales, y luego dividiendo el dataset completo en los 3 datasets mencionados anteriormente.

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [2]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from skimage import io
import torchvision.transforms as transforms

In [3]:
# Para importar las imagenes, crearemos una clase que relaciona la respectiva imagen en su directorio, 
# con la clase a la que pertenece mediante un archivo csv. 

# Para crear inicialmente este archivo csv, hay una utilidad creada que se llama dataset2csv en herramientas.
    
class BasuraDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        #print(self.annotations.iloc[index, 0])
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        if self.annotations.iloc[index, 1] is 'B':
            y_label = 0
        if self.annotations.iloc[index, 1] is 'L':
            y_label = 1
        if self.annotations.iloc[index, 1] is 'T':
            y_label = 2
            
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label)
    
    def get_X(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        
        return image
    
    def get_y(self, index):
        if self.annotations.iloc[index, 1] is 'B':
            y_label = 0
        if self.annotations.iloc[index, 1] is 'L':
            y_label = 1
        if self.annotations.iloc[index, 1] is 'T':
            y_label = 2
        return y_label

In [4]:
import numpy as np

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dataset = BasuraDataset(csv_file = 'dataset.csv', root_dir = 'DatasetReducido') #, transform = transforms.ToTensor()) # Transformar las imagenes a tensor nos permite trabajarlas en pytorch.

In [7]:
X=[]
y=[]

for i in dataset:
    image, y_label = i
    X.append(image)
    y.append(y_label)

In [8]:
X = np.array(X)

In [9]:
X.shape

(2999, 120, 160, 3)

In [10]:
X_train, x_val, y_train, y_val = train_test_split(X,y,test_size=0.33)

In [11]:
#train_set, val_set, test_set = torch.utils.data.random_split(dataset, [1799, 600, 600]) # cerca de 60%, 20%, 20%

Una vez creado y dividido el dataset, partiremos definiendo los hiperparámetros del modelo.

In [12]:
import matplotlib.pyplot as plt

def get_image(data, index):
    image, y_label = data[index] 
    plt.imshow(image.squeeze())
    print('Class: ',y_label)

In [13]:
#get_image(test_set, 0)

In [14]:
#get_image(val_set, 30)

In [15]:
#get_image(train_set, 0)

In [16]:
imag, ind = dataset[0]
(h, w, chan) = imag.shape

In [17]:
batch_size = 1
num_classes = 3
num_epochs = 1
early_stopping = True

In [18]:
#train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=True)

In [55]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        # In: 160x120x3 
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # depth 3 in, 32 out, 3x3 kernel
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1) # depth 32 in, 32 out, 3x3 kernel
        self.pool = nn.MaxPool2d(2,2) # maxpool, reduces dimensionality by 2x2 times
        # Out 1: 80x60x32
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1) # depth 32 in, 64 out, 3x3 kernel
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1) # depth 64 in, 64 out, 3x3 kernel
        # maxpool 2x2
        # Out 2: 40x30x64
        self.conv5 = nn.Conv2d(64, 64, 3, padding=1) # depth 32 in, 64 out, 3x3 kernel
        self.conv6 = nn.Conv2d(64, 64, 3, padding=1) # depth 64 in, 64 out, 3x3 kernel
        
        self.pool2 = nn.MaxPool2d(2,2) # maxpool, reduces dimensionality by 2x2 times
        self.conv7 = nn.Conv2d(64, 5, 3, padding=1) # depth 64 in, 64 out, 3x3 kernel
        # Out 3: 20x15x5
        self.fc1 = nn.Linear(20*15*5, 3) 
        # Out final: 3

    def forward(self, x):
        x = x.view(batch_size, chan, h, w).float()
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(x)
        x = self.pool2(x)
        x = self.conv7(x)
        x = x.view(-1, 20*15*5)
        x = F.relu(self.fc1(x))
        return x


net = Net()

In [56]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [57]:
#enumerate(train_loader, 0)

In [58]:
def one_hot(label):
    t = torch.Tensor([[0,0,0]])
    t[0][label] = 1
    return t

In [59]:
def undo_hot(vector):
    k = 0
    m = 0
    for i in range(len(vector[0])):
        if vector[0][i] > m:
            m = vector[0][i]
            m = i

In [60]:
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(X_train)):
        # get the inputs; data is a list of [inputs, labels]

        # zero the parameter gradients
        optimizer.zero_grad()
        image = X_train[i]
        image = (image - 127/(255))
        # forward + backward + optimize
        outputs = net(torch.tensor(image))
        print(outputs)
        loss = criterion(outputs, torch.tensor([y_train[i]]))
        print(torch.tensor([y_train[i]]))
        #print(loss)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

tensor([[ 0.0000, 10.0341,  3.9698]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.0974, 2.8306]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.0000, 10.3775,  2.2891]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.6383, 3.0626]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 8.3063, 3.1745]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.5167, 3.6700]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.3063, 3.5234]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.8669, 3.0655]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.4998, 8.5638, 3.4666]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.4217, 9.2468, 2.3687]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.4045, 1.0531]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.3612, 2.0197]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.4141, 8.1678, 2.4857]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 8.5470, 1.6962]], grad_fn=<ReluBackward0>

tensor([[0.0000, 9.5978, 3.6825]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.2751, 9.4803, 4.1339]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 10.0650,  3.4473]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.8882, 1.3366]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.9128, 10.0546,  4.7429]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[1.2993, 9.2297, 2.4304]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.0000, 10.3616,  3.5207]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.9525, 4.4056]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.4799, 3.1167]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.9961, 2.1840]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.2989, 4.3424]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.5524, 8.3494, 3.7014]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.0000, 10.4159,  4.3960]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.0000, 10.0367,  1.6914]], grad_fn=<ReluB

tensor([[0.0000, 9.4436, 3.1620]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 11.5904,  3.6512]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.2230, 4.2558]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.3746, 10.4243,  4.1215]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.4702, 8.5700, 3.1948]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.3176, 2.7076]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 11.8333,  3.2082]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0214, 8.6531, 5.0378]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.6402, 3.1813]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.9552, 3.4666]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.0000, 10.7572,  4.2070]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0172, 9.0362, 3.0002]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.4840, 2.0231]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.4437, 4.4823]], grad_fn=<ReluBack

tensor([[ 1.0494, 10.6250,  4.5145]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.1655, 11.0478,  3.7675]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 10.2574,  4.5019]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.5576, 2.0774]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 10.0197,  4.0085]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.4194, 9.6315, 3.4203]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.0000, 10.0906,  4.0948]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.6675, 3.0179]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.6561, 3.2820]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.9475, 3.9597]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.8181, 3.8549]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.5083, 1.7467]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.0170, 11.3005,  3.4546]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.6874, 10.2279,  2.9377]], grad_fn=

tensor([[0.0000, 8.4705, 0.9298]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 8.7059, 2.4036]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.4865, 3.8042]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.5254, 3.1026]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.0000, 10.4351,  2.0502]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 11.3555,  3.9893]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 10.2454,  3.9659]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.2404, 2.3510]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[ 0.2149, 10.5507,  5.0860]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0406, 9.3133, 2.6265]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0573, 8.3310, 4.2155]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.4089, 2.6707]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.5855, 10.0539,  4.4282]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.2440, 9.2311, 3.1595]], grad_fn=<ReluB

...

tensor([[ 0.0000, 11.5032,  3.6298]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.5081, 3.2842]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.4093, 2.7560]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.9015, 1.8983]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.8604, 3.7925]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0975, 9.3901, 2.1690]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.4177, 10.5798,  4.0363]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.7505, 4.0211]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0850, 9.6455, 2.9531]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.2404, 2.6400]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.0003, 2.6670]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.1358, 8.8139, 3.5557]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.9782, 4.1324]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[1.7310, 8.1630, 3.6026]], grad_fn=<ReluBackward0>

tensor([[0.0000, 9.9775, 2.3931]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.1516, 3.6441]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.6104, 3.7468]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.1703, 2.0356]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.7512, 3.4404]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0215, 9.8474, 3.3343]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.1898, 2.4241]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.6743, 4.0350]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.4525, 3.0808]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.3663, 3.9855]], grad_fn=<ReluBackward0>)
tensor([1])
[1,  1600] loss: 0.324
tensor([[0.7093, 9.0474, 2.8303]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.3563, 2.9760]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0884, 9.1900, 3.0120]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.8118, 9.3327, 3.4881]], grad_f

tensor([[0.0000, 9.5818, 2.0654]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.2590, 2.3976]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.7634, 4.0257]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.0726, 3.1704]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.1498, 2.4551]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.4241, 4.1285]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.0363, 2.6935]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.4069, 2.7090]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.0944, 3.2312]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.2313, 2.8106]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[ 0.0000, 10.3288,  3.1456]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.6688, 3.3577]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 1.3020, 10.4107,  4.0917]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.7781, 3.4087]], grad_fn=<ReluBackward0>

tensor([[0.0000, 9.4917, 2.6778]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.3496, 2.4651]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 7.6697, 3.2633]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.6641, 2.5739]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[1.0739, 9.1429, 3.7408]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.3730, 2.8352]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.4936, 3.7086]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0996, 9.0864, 3.1730]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.2731, 3.9026]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 8.8706, 1.7488]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.7691, 0.9055]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 9.5644, 3.1868]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.9457, 3.2211]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.5453, 3.2219]], grad_fn=<ReluBackward0>)
tens

tensor([[ 0.6747, 10.4531,  4.1308]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.1796, 1.3696]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.1461, 9.9634, 3.8728]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.3707, 4.2264]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 8.7486, 2.9955]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.0000, 9.6486, 3.1495]], grad_fn=<ReluBackward0>)
tensor([1])
tensor([[0.0000, 8.8621, 2.4336]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.5904, 3.9364]], grad_fn=<ReluBackward0>)
tensor([0])
tensor([[0.4700, 9.8300, 3.7708]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0000, 9.9074, 4.3268]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 1.2865, 10.9564,  4.3588]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[ 0.0000, 10.5401,  3.5636]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.2964, 9.6419, 3.5651]], grad_fn=<ReluBackward0>)
tensor([2])
tensor([[0.0659, 8.7694, 2.5076]], grad_fn=<ReluBackwar

In [ ]:
val_set

In [ ]:
y_test = net.eval(val_set)

In [ ]:
import 

In [ ]:
import torchvision
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    #img = img // 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (0,1,2)))
    plt.show()

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(labels)

outputs = net(images)

print(outputs)